# Исследование надежности заемщиков


## Наш заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку

### Откроем таблицу и изучим общую информацию о данных

**Импортируем библиотеку pandas. Считаем данные из csv-файла в датафрейм и сохраняем в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Выведем первые 20 строчек датафрейма `data` на экран.**

In [2]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Выведем основную информацию о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Выведем количество пропущенных значений для каждого столбца. Используем комбинацию двух методов.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Выведем перечень уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

**В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

**Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Убедимся, что все пропуски заполнены. Проверим себя и ещё раз выведем количество пропущенных значений для каждого столбца с помощью двух методов.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.**

In [14]:
data.duplicated().sum()

54

In [15]:
data = data.drop_duplicates()

**Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.**

In [16]:
data['education'] = data['education'].str.lower()

### Категоризация данных

**На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 назначим категорию `'E'`, а клиенту, получающему 235000, — `'B'`.**

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создали функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

## Исследуйте данные и ответьте на вопросы

**Задание 19. Есть ли зависимость между количеством детей и возвратом кредита в срок?**

In [22]:
data = data.rename(columns={'debt': 'credit_repayment_status'})


In [23]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,credit_repayment_status,total_income,purpose,total_income_category,purpose_category
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


In [24]:
children_debt = data.groupby('children').agg({'credit_repayment_status':['count','sum']}) # агрегированная функция зависимости просрочки по кредиту от количества детей
print(children_debt)

         credit_repayment_status      
                           count   sum
children                              
0                          14107  1063
1                           4809   444
2                           2052   194
3                            330    27
4                             41     4
5                              9     0


In [25]:
children_fail = children_debt['credit_repayment_status']['sum'] / children_debt['credit_repayment_status']['count'] *100 #конверсия просрочки кредита по количеству детей
print(children_fail.sort_values(ascending=False).round(2))

children
4    9.76
2    9.45
1    9.23
3    8.18
0    7.54
5    0.00
dtype: float64


In [26]:
children_pivot = data.pivot_table(index = ['children'], \
                                columns = ['credit_repayment_status'], \
                                values = 'purpose_category', aggfunc='count')

children_pivot['ratio %'] = children_pivot[1] / (children_pivot[0]+children_pivot[1]) * 100
children_pivot.round(2)

credit_repayment_status,0,1,ratio %
children,,,
0,13044.0,1063.0,7.54
1,4365.0,444.0,9.23
2,1858.0,194.0,9.45
3,303.0,27.0,8.18
4,37.0,4.0,9.76
5,9.0,NaN,NaN


**Вывод:** 
*Заемщики, не имеющие детей, менее склонны к просрочке по выплатам кредита.*
*С увеличением количества детей мы видим и увеличение количества просроченных кредитов, однако, люди с 3 детьми чаще платят в срок, чем люди с 1 ребенком.*


**Задание 20. Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [27]:
family_grouped = data.groupby('family_status').agg({'credit_repayment_status':['count','sum']}) #агг.функция зависимости просрочки по кредиту от семейного положения
print(family_grouped)

                      credit_repayment_status     
                                        count  sum
family_status                                     
Не женат / не замужем                    2796  273
в разводе                                1189   84
вдовец / вдова                            951   63
гражданский брак                         4146  385
женат / замужем                         12266  927


In [28]:
family_debt_fail = family_grouped['credit_repayment_status']['sum'] / family_grouped['credit_repayment_status']['count'] * 100 #конверсия просрочки по сем.положению
print(family_debt_fail.sort_values(ascending=False).round(2))

family_status
Не женат / не замужем    9.76
гражданский брак         9.29
женат / замужем          7.56
в разводе                7.06
вдовец / вдова           6.62
dtype: float64


In [29]:
family_status_pivot = data.pivot_table(index=['family_status'], \
                                       columns = 'credit_repayment_status', \
                                       values='family_status_id', aggfunc='count')
family_status_pivot['ratio %'] = family_status_pivot[1] / (family_status_pivot[0]+family_status_pivot[1]) * 100
family_status_pivot.round(2)

credit_repayment_status,0,1,ratio %
family_status,,,
Не женат / не замужем,2523,273,9.76
в разводе,1105,84,7.06
вдовец / вдова,888,63,6.62
гражданский брак,3761,385,9.29
женат / замужем,11339,927,7.56


**Вывод:** 
*Зависимость есть.*
*Люди* **не в браке или в гражданском браке** *имеют бОльший процент невозвратов в срок. А те, кто развелись или овдовели чаще платят в срок, чем люди в браке*

**Задание 21. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

**Высчитываю конверсию просрочки по уровню дохода**

In [30]:
income_set = data.groupby('total_income_category')['credit_repayment_status'].sum() / \
      data.groupby('total_income_category')['credit_repayment_status'].count() * 100

In [31]:
income_set.sort_values().round(2)

total_income_category
D    6.02
B    7.06
A    8.00
C    8.49
E    9.09
Name: credit_repayment_status, dtype: float64

**Создаю сводную таблицу по уровням дохода, округляю до двух знаков**

In [32]:
total_income_pivot = data.pivot_table(index=['total_income_category'], \
                                      columns = 'credit_repayment_status', \
                                      values='family_status_id', aggfunc='count')

total_income_pivot['ratio %'] = total_income_pivot[1] / (total_income_pivot[0]+total_income_pivot[1]) * 100

total_income_pivot.round(2)


credit_repayment_status,0,1,ratio %
total_income_category,,,
A,23,2,8.00
B,4660,354,7.06
C,14585,1353,8.49
D,328,21,6.02
E,20,2,9.09


**Можно сделать вывод, что хуже всего покатель у людей с очень низкими доходами, но есть проблема по выборке, недостаточно людей в этой группе, чтобы делать какие-то заключения, такая же ситуация у людей с очень высокими доходами. Выраженной зависимости между уровнем дохода и возвратом кредита в срок обнаружить не удалось. Но можно предположить, что лучше всего дела обстоят у людей с доходом чуть выше низкого в группе D**

**Задание 22. Как разные цели кредита влияют на его возврат в срок?**

**Создаю сводную таблицу для разных категорий целей кредита**

In [33]:
credit_purpose_pivot = data.pivot_table(index=['purpose_category'], \
                                        columns=['credit_repayment_status'], \
                                        values='education_id', aggfunc='count')
credit_purpose_pivot['ratio %'] = credit_purpose_pivot[1] / (credit_purpose_pivot[0]+credit_purpose_pivot[1]) * 100
credit_purpose_pivot.round(2)

credit_repayment_status,0,1,ratio %
purpose_category,,,
операции с автомобилем,3881,400,9.34
операции с недвижимостью,9974,780,7.25
получение образования,3620,369,9.25
проведение свадьбы,2141,183,7.87


**Вывод:** 
*Люди с большей вероятностью выплачивают кредит, если он взят на операции с недвижимостью или на свадьбу, нежели на автомобиль или образование*

**Задание 23. Приведите возможные причины появления пропусков в исходных данных.**

*-Ошибки в оборудовании при сборе информации,*

*-Человеческий фактор при заполнении данных*


**Задание 24. Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.**

*Так как значения могут разительно отличаться друг от друга и среднее арифметическое будет не объективно для заполнения пропусков, то применяют медианное значение* 

## Общий вывод.

<div class="alert alert-info">
    
В ходе работы искали зависимость между:

1) Наличием, количеством детей и возвратом кредита в срок - должников с детьми больше, чем бездетных, следовательно, наличие детей негативно влияет на возврат кредита в срок.

2) Семейным положением и возвратом кредита в срок - зависимость есть, чаще всего становятся должниками люди с семейным положением не женат / не замужем.

3) Уровнем дохода и возвратом кредита в срок - выраженной зависимости между уровнем дохода и возвратом кредита в срок обнаружено не было, хоть и зарабатывающие ниже среднего лучше других возвращают долги.

4) Целью кредита и возвратом кредита в срок - зависимость есть, значительная доля должников взяла кредит на автомобиль или образование. На свадьбу или недвижимость - должников меньше.
<b>
    
Подытожив, можно предположить, что надежные заёмщики - это люди: \
    в браке, либо гражданском браке \
    без детей, \
    с доходами ниже среднего, \
    либо которые берут кредит на недвижимость или свадьбу
    
</div>